Creation of a dataset
=====================

This example shows how to use the `pymia.data` package to create a dataset.
The Jupyter notebook can be found at `./examples/data/creation.ipynb`.

<div class="alert alert-info">

Note

To be able to run this example:

- Get the example data by executing `./examples/example-data/pull_example_data.py`.

</div>

The example data consists of 3 tesla head magnetic resonance (MR) images of four unrelated healthy subjects from the [Human Connectome Project (HCP)](https://www.humanconnectome.org/) dataset of healthy volunteers [1].
Each subject will have four 3-D images (MetaImage format `.mha` and Nifty format `.nii.gz`) and demographic information as text file `.txt`.
The images are a T1-weighted, a T2-weighted, a ground truth (GT), and a mask image. The demographic information are age, sex, and grade point average (GPA), all .

The ground truth images contain segmentation of five brain structures:

- 0: background
- 1: Cortical and cerebellar white matter
- 2: Cerebral and cerebellar cortex / grey matter
- 3: Hippocampus
- 4: Amygdala
- 5: Thalamus

Therefore, the example data mimics the problem of medical image segmentation of brain tissues. In this example, you will learn how to create a dataset, which allows us to have easy, flexible, and fast access to our data with the `pymia.data` package.

Import the required modules.

In [1]:
import enum
import glob
import os
import typing

import SimpleITK as sitk
import numpy as np

import pymia.data as data
import pymia.data.conversion as conv
import pymia.data.definition as defs
import pymia.data.creation as crt
import pymia.data.transformation as tfm
import pymia.data.creation.fileloader as file_load

Let us first define an enumeration with the data we will write to the dataset.

In [2]:
class FileTypes(enum.Enum):
    T1 = 1  # The T1-weighted MR image
    T2 = 2  # The T2-weighted MR image
    GT = 3  # The ground truth image
    MASK = 4  # The foreground mask
    AGE = 5  # The age
    GPA = 6  # The GPA
    SEX = 7  # The sex

Next, we define a subject. Each subject will have two structural MR images (T1w, T2w), one label image (ground truth), a mask, two numericals (age and GPA), and the sex (a character "m" or "w").

In [3]:
class Subject(data.SubjectFile):

    def __init__(self, subject: str, files: dict):
        super().__init__(subject,
                         images={FileTypes.T1.name: files[FileTypes.T1], FileTypes.T2.name: files[FileTypes.T2]},
                         labels={FileTypes.GT.name: files[FileTypes.GT]},
                         mask={FileTypes.MASK.name: files[FileTypes.MASK]},
                         numerical={FileTypes.AGE.name: files[FileTypes.AGE], FileTypes.GPA.name: files[FileTypes.GPA]},
                         sex={FileTypes.SEX.name: files[FileTypes.SEX]})
        self.subject_path = files.get(subject, '')

We now collect the subjects, and initialize a `Subject` holding paths to each of the data.

In [4]:
data_dir = '../example-data'

# get subjects
subject_dirs = [subject_dir for subject_dir in glob.glob(os.path.join(data_dir, '*')) if os.path.isdir(subject_dir) and os.path.basename(subject_dir).startswith('Subject')]
sorted(subject_dirs)

# the keys of the data to write to the dataset
keys = [FileTypes.T1, FileTypes.T2, FileTypes.GT, FileTypes.MASK, FileTypes.AGE, FileTypes.GPA, FileTypes.SEX]

subjects = []
# for each subject on file system, initialize a Subject object
for subject_dir in subject_dirs:
    id_ = os.path.basename(subject_dir)

    file_dict = {id_: subject_dir}  # init dict with id_ pointing to the path of the subject
    for file_key in keys:
        if file_key == FileTypes.T1:
            file_name = f'{id_}_T1.mha'
        elif file_key == FileTypes.T2:
            file_name = f'{id_}_T2.mha'
        elif file_key == FileTypes.GT:
            file_name = f'{id_}_GT.mha'
        elif file_key == FileTypes.MASK:
            file_name = f'{id_}_MASK.nii.gz'
        elif file_key == FileTypes.AGE or file_key == FileTypes.GPA or file_key == FileTypes.SEX:
            file_name = f'{id_}_demographic.txt'
        else:
            raise ValueError('Unknown key')

        file_dict[file_key] = os.path.join(subject_dir, file_name)

    subjects.append(Subject(id_, file_dict))

Then, we define a `LoadData` class. We load the structural MR images (T1w and T2w) as float and the other images as int. The age, sex, and GPA are loaded from the text file.

In [5]:
class LoadData(file_load.Load):

    def __call__(self, file_name: str, id_: str, category: str, subject_id: str) -> \
            typing.Tuple[np.ndarray, typing.Union[conv.ImageProperties, None]]:
        if id_ == FileTypes.AGE.name:
            with open(file_name, 'r') as f:
                value = np.asarray([int(f.readline().split(':')[1].strip())])
                return value, None
        if id_ == FileTypes.GPA.name:
            with open(file_name, 'r') as f:
                value = np.asarray([float(f.readlines()[1].split(':')[1].strip())])
                return value, None
        if id_ == FileTypes.SEX.name:
            with open(file_name, 'r') as f:
                value = np.array(f.readlines()[2].split(':')[1].strip())
                return value, None

        if category == defs.KEY_IMAGES:
            img = sitk.ReadImage(file_name, sitk.sitkFloat32)
        else:
            # this is the ground truth (defs.KEY_LABELS) and mask, which will be loaded as unsigned integer
            img = sitk.ReadImage(file_name, sitk.sitkUInt8)

        # return both the image intensities as np.ndarray and the properties of the image
        return sitk.GetArrayFromImage(img), conv.ImageProperties(img)


Finally, we can use a writer to create the HDF5 dataset by passing the list of `Subject`s and the `LoadData` to a `SubjectFileTraverser`. For the structural MR images, we also apply an intensity normalization.

In [6]:
hdf_file = '../example-data/example-dataset.h5'

# remove the "old" dataset if it exists
if os.path.exists(hdf_file):
    os.remove(hdf_file)

with crt.get_writer(hdf_file) as writer:
    # initialize the callbacks that will actually write the data to the dataset file
    callbacks = crt.get_default_callbacks(writer)

    # add a transform to normalize the structural MR images
    transform = tfm.IntensityNormalization(loop_axis=3, entries=(defs.KEY_IMAGES, ))

    # run through the subject files (loads them, applies transformations, and calls the callback for writing them)
    traverser = crt.SubjectFileTraverser()
    traverser.traverse(subjects, callback=callbacks, load=LoadData(), transform=transform)

start dataset creation
[1/4] Subject_1
[2/4] Subject_2
[3/4] Subject_3
[4/4] Subject_4
dataset creation finished


This should now have created a `example-dataset.h5` in the directory `./examples/example-data`. By using a HDF5 viewer like [HDF Compass](https://support.hdfgroup.org/projects/compass/) or [HDFView](https://www.hdfgroup.org/downloads/hdfview/), we can inspect the dataset. It should look similar to the figure below.

![The dataset structure.](./images/fig-dataset.png)

References
----------

[1] Van Essen, D.C., Smith, S.M., Barch, D.M., Behrens, T.E., Yacoub, E., Ugurbil, K. and Wu-Minn HCP Consortium, 2013. The WU-Minn human connectome project: an overview. Neuroimage, 80, pp.62-79.